# Coffee Shop Project: Austin, Texas
## Capstone Project - The Battle of Neighborhoods (Week 2)
### By: Sanjit Madireddi

##### Importing the Modules

In [1]:
import requests
import pandas as pd
import numpy as np
! pip install folium
import folium 

##### Inputting Foursquare Credentials

In [2]:
CLIENT_ID = 'XDR43DJ3GFCUZ5W4HEVAEQBOKG0KF14JU1H2T2S5NBEBF40Z' # your Foursquare ID
CLIENT_SECRET = '22NNACFJDVQ2MTOKFZJKI421M2KXUKUFCZOOZUGD3SHJTD1W' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 40
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XDR43DJ3GFCUZ5W4HEVAEQBOKG0KF14JU1H2T2S5NBEBF40Z
CLIENT_SECRET:22NNACFJDVQ2MTOKFZJKI421M2KXUKUFCZOOZUGD3SHJTD1W


##### Obtaining the required Dataset for the Project from Foursquare

In [3]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "coffee",
    "near": "Austin",
    "radius": 1000,
    "limit": 50}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [4]:
data = data.json()["response"]
data.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [5]:
data["headerLocationGranularity"], data["headerLocation"], data["headerFullLocation"]

('city', 'Austin', 'Austin')

In [6]:
data["suggestedBounds"], data["totalResults"]

({'ne': {'lat': 30.27204329253869, 'lng': -97.7375035004786},
  'sw': {'lat': 30.25936485668744, 'lng': -97.75278822323023}},
 54)

In [7]:
data["geocode"]

{'what': '',
 'where': 'austin',
 'center': {'lat': 30.26715, 'lng': -97.74306},
 'displayString': 'Austin, TX, United States',
 'cc': 'US',
 'geometry': {'bounds': {'ne': {'lat': 30.516862999999997, 'lng': -97.561489},
   'sw': {'lat': 30.098658999999998, 'lng': -97.938383}}},
 'slug': 'austin-texas',
 'longId': '72057594042599590'}

In [8]:
data["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [9]:
data["groups"][0]["type"], data["groups"][0]["name"]

('Recommended Places', 'recommended')

In [10]:
items = data["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 50


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '50981188e4b0f94e062c8664',
  'name': 'Houndstooth Coffee',
  'location': {'address': '401 Congress Ave',
   'crossStreet': 'at 4th St',
   'lat': 30.266194,
   'lng': -97.743025,
   'labeledLatLngs': [{'label': 'display',
     'lat': 30.266194,
     'lng': -97.743025}],
   'postalCode': '78701',
   'cc': 'US',
   'city': 'Austin',
   'state': 'TX',
   'country': 'United States',
   'formattedAddress': ['401 Congress Ave (at 4th St)',
    'Austin, TX 78701',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d1e0931735',
    'name': 'Coffee Shop',
    'pluralName': 'Coffee Shops',
    'shortName': 'Coffee Shop',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-5-50981188e4b0f94e06

In [11]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    print(location)
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address =  ''
    if hasattr(location, 'address'):
      address = location['address']
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i cafes" % len(df))
df.head()

{'address': '401 Congress Ave', 'crossStreet': 'at 4th St', 'lat': 30.266194, 'lng': -97.743025, 'labeledLatLngs': [{'label': 'display', 'lat': 30.266194, 'lng': -97.743025}], 'postalCode': '78701', 'cc': 'US', 'city': 'Austin', 'state': 'TX', 'country': 'United States', 'formattedAddress': ['401 Congress Ave (at 4th St)', 'Austin, TX 78701', 'United States']}
{'address': '609 Congress Ave', 'crossStreet': 'btw 6th St. & 7th St.', 'lat': 30.268378003234933, 'lng': -97.74267845007364, 'labeledLatLngs': [{'label': 'display', 'lat': 30.268378003234933, 'lng': -97.74267845007364}], 'postalCode': '78701', 'cc': 'US', 'city': 'Austin', 'state': 'TX', 'country': 'United States', 'formattedAddress': ['609 Congress Ave (btw 6th St. & 7th St.)', 'Austin, TX 78701', 'United States']}
{'address': '212 E 6th St', 'crossStreet': 'at Brazos St', 'lat': 30.267601825957104, 'lng': -97.7407854795456, 'labeledLatLngs': [{'label': 'display', 'lat': 30.267601825957104, 'lng': -97.7407854795456}], 'postalCo

,uid,name,shortname,address,postalcode,lat,lng
0,50981188e4b0f94e062c8664,Houndstooth Coffee,Coffee Shop,,78701,30.266194,-97.743025
1,4d21da6dd7b0b1f7c980209f,Royal Blue Grocery,Grocery Store,,78701,30.268378,-97.742678
2,56170c17498ecd6b0a1e7930,Voodoo Doughnut,Donuts,,78701,30.267602,-97.740785
3,423a1a00f964a5202f201fe3,"Halcyon Coffee, Bar & Lounge",Coffee Shop,,78701,30.266935,-97.745717
4,575825ea498e6d816e1142cb,Caffé Medici,Coffee Shop,,78701,30.270119,-97.742154


In [12]:
mid_austin = data["geocode"]["center"]
mid_austin

{'lat': 30.26715, 'lng': -97.74306}

##### Plotting coffee shops as points in the city of Austin, Texas

In [13]:
from folium import plugins


austin_map = folium.Map(location=[30.26715, -97.74306], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.75,
            parse_html=False).add_to(austin_map)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
austin_map.add_child(plugins.HeatMap(hm_data))

austin_map

##### Plotting the Clustered Map to find the optimal location  

In [14]:
lat = 30.26715
lng = -97.74306
map_DaNang = folium.Map(location=[lat, lng], zoom_start=14)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="Here!",
    color='purple',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.75,
    parse_html=False).add_to(austin_map)
austin_map